<a href="https://colab.research.google.com/github/Guillerumo/Estudando_Python/blob/main/Web_Scrapping_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalamos librerías necesarias (requests y beautifulsoup suelen venir, pero aseguramos)
!pip install requests beautifulsoup4 pandas textblob

import requests
from bs4 import BeautifulSoup
import pandas as pd
from textblob import TextBlob # Esta será nuestra "AI" gratuita
import time

print("Librerías listas. ¡Vamos!")

Librerías listas. ¡Vamos!


In [ ]:
#URL Objetivo
url = "http://books.toscrape.com/"

#Hacemos la peticion
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

#Lista para guardar los datos crudos
libros_data = []

#Buscamos todos los artículos (libros)
articles = soup.find_all('article', class_= 'product_pod')

for book in articles:
  #Extracción del título
  title = book.h3.a['title']

  #Extracción del precio
  price_text = book.find('p', class_='price_color').text

  #Extracción de disponibilidad
  availability = book.find('p', class_= 'instock availability').text.strip()

  libros_data.append({
      'Titulo': title,
      'Precio_Texto': price_text,
      'Disponibilidad': availability
  })


#Convertimos en Data Frame
df = pd.DataFrame(libros_data)
print(f"Extracción completada! Se bajaron {len(df)} libros")
df.head()

Extracción completada! Se bajaron 20 libros


,Titulo,Precio_Texto,Disponibilidad
0,A Light in the Attic,Â£51.77,In stock
1,Tipping the Velvet,Â£53.74,In stock
2,Soumission,Â£50.10,In stock
3,Sharp Objects,Â£47.82,In stock
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock


In [ ]:
#1. Limpieza: Quitar el símbolo '£' y convertir a número
#Usamos str.replace y astype (float)

df['Precio_Num'] = df['Precio_Texto'].str.replace(r'[^\d.]', '', regex=True).astype(float)

#2. Validación: Verificar si hay precios menores a 0 o datos vacíos
#Creamos una columna de "Status" para simular un control de Calidad

def validar_fila(row):
  if row['Precio_Num'] <= 0:
    return "Error: Precio Inválido"
  if row['Titulo'] == "":
    return "Error: Sin Título"
  return "Válido"

df['Data_Quality_Check'] = df.apply(validar_fila, axis = 1)

#Filtramos solo los válidos para el siguiente paso
df_validos = df[df['Data_Quality_Check'] == "Válido"].copy()

print("Datos validados y limpios:")
df_validos[['Titulo', 'Precio_Num', 'Data_Quality_Check']].head()

Datos validados y limpios:


,Titulo,Precio_Num,Data_Quality_Check
0,A Light in the Attic,51.77,Válido
1,Tipping the Velvet,53.74,Válido
2,Soumission,50.10,Válido
3,Sharp Objects,47.82,Válido
4,Sapiens: A Brief History of Humankind,54.23,Válido


In [ ]:
#Función de enriquecimiento
def enriquecer_data(texto):
  #creamos un objeto TextBlob
  blob = TextBlob(texto)
  #Obtenemos la polaridad (-1 es muy negativo, +1 es muy positivo)
  sentimiento = blob.sentiment.polarity

  if sentimiento > 0:
    return "Título positivo"
  elif sentimiento < 0:
    return "Título Negativo"
  else:
    return "Título Neutral"

#Aplicamos la función a cada título
df_validos['AI_Sentiment_Analysis'] = df_validos['Titulo'].apply(enriquecer_data)

print("Datos Enriquecidos con IA!")
df_validos[['Titulo', 'AI_Sentiment_Analysis']].head()

Datos Enriquecidos con IA!


,Titulo,AI_Sentiment_Analysis
0,A Light in the Attic,Título positivo
1,Tipping the Velvet,Título Neutral
2,Soumission,Título Neutral
3,Sharp Objects,Título Negativo
4,Sapiens: A Brief History of Humankind,Título Neutral


In [ ]:
nombre_archivo = "libros_enriquecidos.csv"
df_validos.to_csv(nombre_archivo, index=False)

print(f"Archivo guardado exitosamente: {nombre_archivo}")

Archivo guardado exitosamente: libros_enriquecidos.csv
